In [1]:
!pip install textstat
!pip install pyspellchecker
!pip install vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 78.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 5.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from textstat import textstat
import spacy
import torch
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score
import os
import pickle
from sklearn.preprocessing import StandardScaler
from spellchecker import SpellChecker
from collections import Counter
from sklearn.preprocessing import minmax_scale
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import torch
from torch.utils.data import Dataset, DataLoader

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
train_data = None
train_dir = "/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv"
processed_data_dir = '/kaggle/input/aes-data-more-feature/aes_train_data_processed'

if os.path.exists(processed_data_dir):
    print('processed data exists')
    train_data = pickle.load(open(processed_data_dir, 'rb'))
else:
    train_data = pd.read_csv(train_dir)

processed data exists


In [5]:
train_data.head()

,essay_id,full_text,score,flesch_reading_ease,flesch_kincaid_grade,smog_index,coleman_liau_index,automated_readability_index,dale_chall_readability_score,difficult_words,...,std_neutral,num_words,num_sentences,avg_sentence_length,readability,num_clauses,avg_clause_length,unique_words,pos_tags,pos_counts
0,000d118,Many people have car where they live. The thin...,3,57.98,14.7,11.7,8.19,18.3,8.74,60,...,0.090188,545,13,41.923077,57.98,13,41.923077,248,"[(Many, JJ), (people, NNS), (have, VBP), (car,...","{'JJ': 33, 'NNS': 36, 'VBP': 28, 'NN': 74, 'WR..."
1,000fe60,I am a scientist at NASA that is discussing th...,3,87.55,5.4,6.8,4.99,6.2,6.31,24,...,0.074660,371,19,19.526316,87.55,20,18.550000,168,"[(I, PRP), (am, VBP), (a, DT), (scientist, NN)...","{'PRP': 24, 'VBP': 16, 'DT': 40, 'NN': 43, 'IN..."
2,001ab80,People always wish they had the same technolog...,4,65.15,9.9,11.5,8.94,11.6,7.24,67,...,0.100791,605,24,25.208333,65.15,25,24.200000,243,"[(People, NNS), (always, RB), (wish, VBP), (th...","{'NNS': 39, 'RB': 51, 'VBP': 23, 'PRP': 20, 'V..."
3,001bdc0,"We all heard about Venus, the planet without a...",4,58.32,10.4,13.2,10.97,12.9,8.50,78,...,0.126816,511,21,24.333333,58.32,21,24.333333,241,"[(We, PRP), (all, DT), (heard, VBP), (about, I...","{'PRP': 19, 'DT': 49, 'VBP': 10, 'IN': 63, 'NN..."
4,002ba53,"Dear, State Senator\n\nThis is a letter to arg...",3,54.66,11.8,13.0,10.57,13.9,7.79,55,...,0.150539,418,15,27.866667,54.66,16,26.125000,156,"[(Dear, NNP), (,, ,), (State, NNP), (Senator, ...","{'NNP': 34, ',': 13, 'DT': 67, 'VBZ': 24, 'NN'..."


In [6]:
def textstat_features(text):
    features = {}
    features['flesch_reading_ease'] = textstat.flesch_reading_ease(text)
    features['flesch_kincaid_grade'] = textstat.flesch_kincaid_grade(text)
    features['smog_index'] = textstat.smog_index(text)
    features['coleman_liau_index'] = textstat.coleman_liau_index(text)
    features['automated_readability_index'] = textstat.automated_readability_index(
        text)
    features['dale_chall_readability_score'] = textstat.dale_chall_readability_score(
        text)
    features['difficult_words'] = textstat.difficult_words(text)
    features['linsear_write_formula'] = textstat.linsear_write_formula(text)
    features['gunning_fog'] = textstat.gunning_fog(text)
    features['text_standard'] = textstat.text_standard(text, float_output=True)
    features['spache_readability'] = textstat.spache_readability(text)
    features['mcalpine_eflaw'] = textstat.mcalpine_eflaw(text)
    features['reading_time'] = textstat.reading_time(text)
    features['syllable_count'] = textstat.syllable_count(text)
    features['lexicon_count'] = textstat.lexicon_count(text)
    features['monosyllabcount'] = textstat.monosyllabcount(text)

    return features

In [7]:
def spelling_features(text):
    spell = SpellChecker()

    features = {}

    words = nltk.word_tokenize(text)
    misspelled = spell.unknown(words)

    misspelled_count = len(misspelled)
    misspelled_ratio = misspelled_count / len(words)

    features['misspelled_count'] = misspelled_count
    features['misspelled_count'] = misspelled_ratio

    return features

In [8]:
def extract_linguistic_features(text):

    doc = nlp(text)
    features = {}

    # tense features
    tenses = [i.morph.get("Tense") for i in doc]
    tenses = [i[0] for i in tenses if i]
    tense_counts = Counter(tenses)
    features['past_tense_ratio'] = tense_counts.get(
        "Past", 0) / (tense_counts.get("Pres", 0) + tense_counts.get("Past", 0) + 1e-5)
    features['present_tense_ratio'] = tense_counts.get(
        "Pres", 0) / (tense_counts.get("Pres", 0) + tense_counts.get("Past", 0) + 1e-5)

    # len features

    features['word_count'] = len(doc)
    features['sentence_count'] = len([sentence for sentence in doc.sents])
    features['words_per_sentence'] = features['word_count'] / \
        features['sentence_count']
    features['std_words_per_sentence'] = np.std(
        [len(sentence) for sentence in doc.sents])

    features['unique_words'] = len(set([token.text for token in doc]))
    features['lexical_diversity'] = features['unique_words'] / \
        features['word_count']

    paragraph = text.split('\n\n')

    features['paragraph_count'] = len(paragraph)

    features['avg_chars_by_paragraph'] = np.mean(
        [len(paragraph) for paragraph in paragraph])
    features['avg_words_by_paragraph'] = np.mean(
        [len(nltk.word_tokenize(paragraph)) for paragraph in paragraph])
    features['avg_sentences_by_paragraph'] = np.mean(
        [len(nltk.sent_tokenize(paragraph)) for paragraph in paragraph])

    # sentiment features
    analyzer = SentimentIntensityAnalyzer()
    sentences = nltk.sent_tokenize(text)

    compound_scores, negative_scores, positive_scores, neutral_scores = [], [], [], []
    for sentence in sentences:
        scores = analyzer.polarity_scores(sentence)
        compound_scores.append(scores['compound'])
        negative_scores.append(scores['neg'])
        positive_scores.append(scores['pos'])
        neutral_scores.append(scores['neu'])

    features["mean_compound"] = np.mean(compound_scores)
    features["mean_negative"] = np.mean(negative_scores)
    features["mean_positive"] = np.mean(positive_scores)
    features["mean_neutral"] = np.mean(neutral_scores)

    features["std_compound"] = np.std(compound_scores)
    features["std_negative"] = np.std(negative_scores)
    features["std_positive"] = np.std(positive_scores)
    features["std_neutral"] = np.std(neutral_scores)

    return features

In [9]:
def extract_features(text):
    tokens = word_tokenize(text)
    pos_tags = pos_tag(tokens)

    # Length-based features
    num_words = len(tokens)
    num_sentences = textstat.sentence_count(text)
    avg_sentence_length = num_words / num_sentences if num_sentences != 0 else 0

    # Readability features
    readability = textstat.flesch_reading_ease(text)

    # Text complexity
    doc = nlp(text)
    num_clauses = sum([1 for token in doc if token.dep_ == 'ROOT'])
    avg_clause_length = num_words / num_clauses if num_clauses != 0 else 0

    # Text variation
    unique_words = len(set(tokens))
    pos_counts = nltk.FreqDist(tag for (word, tag) in pos_tags)


    features = {
        'num_words': num_words,
        'num_sentences': num_sentences,
        'avg_sentence_length': avg_sentence_length,
        'readability': readability,
        'num_clauses': num_clauses,
        'avg_clause_length': avg_clause_length,
        'unique_words': unique_words,
        'pos_tags': pos_tags,
        'pos_counts': dict(pos_counts),
    }


    return features





In [10]:
not_feature_list = ['essay_id', 'full_text', 'score', 'pos_counts', 'pos_tags']


def flatten_features(train_data):

    # Extract the feature columns
    feature_columns = [
        col for col in train_data.columns if col not in not_feature_list]

    print(feature_columns)
    # Flatten the features for each row
    flattened_features = train_data[feature_columns].values.tolist()

    # Convert each list of features to a PyTorch tensor
    flattened_features_tensors = [torch.tensor(
        features, dtype=torch.float32) for features in flattened_features]

    # Add the new column to the DataFrame
    train_data['flattened_features'] = flattened_features_tensors

    return train_data

In [11]:
def extract_all_feature(train_data):

    print('extracting all features')
    train_data['textstat_features'] = train_data['full_text'].apply(
        textstat_features)
    train_textstat = pd.DataFrame(train_data['textstat_features'].tolist())
    train_data = train_data.drop(columns=['textstat_features'])
    train_data = pd.concat([train_data, train_textstat], axis=1)

    train_data['spelling_features'] = train_data['full_text'].apply(
        spelling_features)
    spell_check_df = pd.DataFrame(train_data['spelling_features'].tolist(), columns=[
                                  'misspelled_count', 'misspelled_ratio'])
    train_data = train_data.drop(columns=['spelling_features'])
    train_data = pd.concat([train_data, spell_check_df], axis=1)

    train_data['linguistic_features'] = train_data['full_text'].apply(
        extract_linguistic_features)
    train_linguistic = pd.json_normalize(train_data['linguistic_features'])
    train_data = train_data.drop(columns=['linguistic_features'])
    train_data = pd.concat([train_data, train_linguistic], axis=1)

    train_data['core_features'] = train_data['full_text'].apply(
        extract_features)
    core_df = pd.DataFrame(train_data['core_features'].tolist())
    train_data = train_data.drop(columns=['core_features'])
    train_data = pd.concat([train_data, core_df], axis=1)

    return train_data

In [12]:
def select_features(train_data, features=[]):
    train_data = train_data.drop(columns=list(
        set(train_data.columns)-set(features)))
    return train_data


def normalize_features(train_data, features_to_normalize=[]):
    train_data[features_to_normalize] = minmax_scale(
        train_data[features_to_normalize])
    return train_data

In [13]:
features_to_use = [
    'flesch_reading_ease',
    'flesch_kincaid_grade',
    'smog_index',
    'coleman_liau_index',
    'automated_readability_index',
    'dale_chall_readability_score',
    'difficult_words',
    'linsear_write_formula',
    'gunning_fog',
    'text_standard',
    'spache_readability',
    'mcalpine_eflaw',
    'reading_time',
    'syllable_count',
    'lexicon_count',
    'monosyllabcount',
    'misspelled_count',
    'misspelled_count',
    'past_tense_ratio',
    'present_tense_ratio',
    'word_count',
    'sentence_count',
    'words_per_sentence',
    'std_words_per_sentence',
    'unique_words',
    'lexical_diversity',
    'paragraph_count',
    'avg_chars_by_paragraph',
    'avg_words_by_paragraph',
    'avg_sentences_by_paragraph',
    'num_words',
    'num_sentences',
    'avg_sentence_length',
    'readability',
    'num_clauses',
    'avg_clause_length',
    'unique_words',
    "mean_compound",
    "mean_negative",
    "mean_positive",
    "mean_neutral",
    "std_compound",
    "std_negative",
    "std_positive",
    "std_neutral",
]

In [14]:
if not (os.path.exists(processed_data_dir)):
    train_data = extract_all_feature(train_data)
    train_data.to_csv('aes_train_data_processed.csv')
    pickle.dump(train_data, open("aes_train_data_processed", "wb"))


train_data = select_features(train_data, features_to_use+not_feature_list)
train_data = normalize_features(train_data, features_to_use)
train_data = flatten_features(train_data)

['flesch_reading_ease', 'flesch_kincaid_grade', 'smog_index', 'coleman_liau_index', 'automated_readability_index', 'dale_chall_readability_score', 'difficult_words', 'linsear_write_formula', 'gunning_fog', 'text_standard', 'spache_readability', 'mcalpine_eflaw', 'reading_time', 'syllable_count', 'lexicon_count', 'monosyllabcount', 'misspelled_count', 'past_tense_ratio', 'present_tense_ratio', 'word_count', 'sentence_count', 'words_per_sentence', 'std_words_per_sentence', 'unique_words', 'lexical_diversity', 'paragraph_count', 'avg_chars_by_paragraph', 'avg_words_by_paragraph', 'avg_sentences_by_paragraph', 'mean_compound', 'mean_negative', 'mean_positive', 'mean_neutral', 'std_compound', 'std_negative', 'std_positive', 'std_neutral', 'num_words', 'num_sentences', 'avg_sentence_length', 'readability', 'num_clauses', 'avg_clause_length', 'unique_words']


In [15]:
# Create a dictionary for POS tags
all_pos_tags = set()
for i in train_data['pos_counts']:
    all_pos_tags.update(i.keys())


pos2idx = {tag: idx for idx, tag in enumerate(all_pos_tags)}

In [16]:
class EssayDataset(Dataset):
    def __init__(self, data, pos2idx):
        self.data = data
        self.pos2idx = pos2idx

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        essay = self.data.iloc[idx]
        l = []
        for token, pos_tag in essay['pos_tags']:
            l.append(self.pos2idx.get(pos_tag, 0))
        pos_tags = torch.tensor(l, dtype=torch.long)
        flattened_features = essay['flattened_features']
        score = torch.tensor(essay['score'], dtype=torch.float32)
        return pos_tags, flattened_features, score


def collate_fn(batch):
    pos_tags, features, scores = zip(*batch)

    pos_tags_padded = nn.utils.rnn.pad_sequence(
        pos_tags, batch_first=True, padding_value=0).to(device)

    # Stack feature vectors directly (assuming fixed size)
    features_padded = torch.stack(features).to(device)

    scores = torch.stack(scores).to(device)
    return pos_tags_padded, features_padded, scores

In [17]:

class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()
        self.attention = nn.Linear(hidden_dim, 1, bias=False)

    def forward(self, lstm_output):
        attn_weights = F.softmax(self.attention(lstm_output), dim=1)
        context = torch.bmm(attn_weights.transpose(1, 2), lstm_output)
        return context.squeeze(1), attn_weights


class EssayScoringModel(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, feature_dim, dropout_prob):
        super(EssayScoringModel, self).__init__()
        self.conv1 = nn.Conv1d(embedding_dim, hidden_dim,
                               kernel_size=5, padding='same')
        self.lstm = nn.LSTM(hidden_dim, hidden_dim,
                            batch_first=True, bidirectional=True)
        self.lstm_attention = Attention(hidden_dim*2)
        self.conv1_attention = Attention(hidden_dim)
        self.dropout = nn.Dropout(dropout_prob)
        self.fc1 = nn.Linear(hidden_dim * 2 + feature_dim,
                             256)  # Increased capacity
        self.fc2 = nn.Linear(256, 128)  # Added another fully connected layer
        self.fc3 = nn.Linear(128, 1)

    def forward(self, pos_tags, features):

        # Embedding layer
        # Shape: (batch_size, seq_length, num_classes)
        embeds = F.one_hot(pos_tags, num_classes=50).float()

        # Convolutional layer
        # Shape: (batch_size, hidden_dim, seq_length)
        conv_out = F.relu(self.conv1(embeds.transpose(1, 2)))
        conv_out = self.dropout(conv_out)

        # Attention pooling after convolution
        attn_conv_out = self.conv1_attention(conv_out.transpose(1, 2))[0]  # Shape: (batch_size, hidden_dim)

        # LSTM layer
        # Shape: (batch_size, 1, hidden_dim)
        lstm_out, _ = self.lstm(attn_conv_out.unsqueeze(1))
        lstm_out = self.dropout(lstm_out)

        # Attention pooling after LSTM
        # Shape: (batch_size, hidden_dim * 2)
        attn_lstm_out = self.lstm_attention(lstm_out)[0]

        # Concatenate LSTM output with additional features
        # Shape: (batch_size, hidden_dim * 2 + feature_dim)
        combined = torch.cat((attn_lstm_out, features), dim=1)

        x = F.relu(self.fc1(combined))  # Shape: (batch_size, 256)
        x = self.dropout(x)
        x = F.relu(self.fc2(x))  # Shape: (batch_size, 128)
        x = self.fc3(x)  # Shape: (batch_size, 1)
        x = torch.sigmoid(x) * 5 + 1  # scores are between 1 and 6

        return x

In [18]:
# Training and validation split
train_data, val_data = train_test_split(train_data, test_size=0.2)
train_dataset = EssayDataset(train_data, pos2idx)
val_dataset = EssayDataset(val_data, pos2idx)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=64,
                          shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=64,
                        shuffle=False, collate_fn=collate_fn)

In [19]:
class ModelParameters:
    def __init__(self, train_data):
        self.embedding_dim = 50  # Embedding output dimension
        self.hidden_dim = 100  # number of filters (conv)
        self.feature_dim = len(train_data['flattened_features'].iloc[0])
        self.dropout_prob = 0.5


model_parameters = ModelParameters(train_data)

# Initialize model, loss function, and optimizer
embedding_dim = model_parameters.embedding_dim
hidden_dim = model_parameters.hidden_dim  
feature_dim = model_parameters.feature_dim
dropout_prob = model_parameters.dropout_prob

model = EssayScoringModel(embedding_dim,
                          hidden_dim, feature_dim, dropout_prob).to(device)
criterion = nn.MSELoss()
optimizer = optim.RMSprop(model.parameters(), lr=0.001)

In [20]:
lowest_val_loss = 10**5

# Training loop
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for pos_tags, features, scores in train_loader:
        pos_tags, featuers, scores = pos_tags.to(
            device), features.to(device), scores.to(device)
        optimizer.zero_grad()
        outputs = model(pos_tags, features).squeeze()
        loss = criterion(outputs, scores)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch {epoch + 1}')
    print(f'Train Loss: {running_loss / len(train_loader)}')

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for pos_tags, features, scores in val_loader:
            pos_tags, featuers, scores = pos_tags.to(
                device), features.to(device), scores.to(device)
            outputs = model(pos_tags, features).squeeze()
            loss = criterion(outputs, scores)
            val_loss += loss.item()

    print(f'Valid Loss: {val_loss / len(val_loader)}')

    # save best model weights
    if val_loss / len(val_loader) <= lowest_val_loss:
        lowest_val_loss = val_loss / len(val_loader)
        torch.save(model.state_dict(), 'essay_scoring_best_model.pth')
        print('saved best model')

    print('\n')

Epoch 1
Train Loss: 0.5914397694273479
Valid Loss: 0.5073619977994399
saved best model


Epoch 2
Train Loss: 0.49207359222772484
Valid Loss: 0.4811168746514754
saved best model


Epoch 3
Train Loss: 0.4676665589556716
Valid Loss: 0.4605275159532374
saved best model


Epoch 4
Train Loss: 0.4478749432322067
Valid Loss: 0.4492049986665899
saved best model


Epoch 5
Train Loss: 0.4362673667444062
Valid Loss: 0.4293244031342593
saved best model


Epoch 6
Train Loss: 0.4291763000087255
Valid Loss: 0.45962025929581035


Epoch 7
Train Loss: 0.42292727179790973
Valid Loss: 0.5373822629451752


Epoch 8
Train Loss: 0.4126416436538169
Valid Loss: 0.42298842235045
saved best model


Epoch 9
Train Loss: 0.4086810921743718
Valid Loss: 0.4181366416541013
saved best model


Epoch 10
Train Loss: 0.40302330186839475
Valid Loss: 0.4728509073907679


Epoch 11
Train Loss: 0.40065912105795426
Valid Loss: 0.4134326793930747
saved best model


Epoch 12
Train Loss: 0.39602716088569656
Valid Loss: 0.434494864127

In [21]:
def evaluate_model(model, loader):
    model.eval()
    all_scores = []
    all_preds = []
    with torch.no_grad():
        for pos_tags, features, scores in loader:
            outputs = model(pos_tags, features).squeeze()
            all_scores.extend(scores.tolist())
            all_preds.extend(outputs.tolist())

    all_scores = [round(score) for score in all_scores]
    all_preds = [round(pred) for pred in all_preds]

    qwk_score = cohen_kappa_score(all_scores, all_preds, weights='quadratic')
    return qwk_score


# Load and Evaluate best model
model.load_state_dict(torch.load(
    '/kaggle/working/essay_scoring_best_model.pth'))

qwk_score = evaluate_model(model, val_loader)
print(f'Quadratic Weighted Kappa: {qwk_score}')

Quadratic Weighted Kappa: 0.7659923925088844


In [22]:
pickle.dump(pos2idx, open("pos2idx", "wb"))
pickle.dump(model_parameters, open("model_parameters", "wb"))

In [23]:
# !pip download textstat -d ./textstat/
# !pip download vaderSentiment -d ./vaderSentiment/
# !pip download pyspellchecker -d ./pyspellchecker/

# import os
# from zipfile import ZipFile

# dirName = "./"
# zipName = "packages.zip"

# # Create a ZipFile Object
# with ZipFile(zipName, 'w') as zipObj:
#     # Iterate over all the files in directory
#     for folderName, subfolders, filenames in os.walk(dirName):
#         for filename in filenames:
#             if (filename != zipName):
#                 # create complete filepath of file in directory
#                 filePath = os.path.join(folderName, filename)
#                 # Add file to zip
#                 zipObj.write(filePath)